#### 개정 일자: 2020.11.02


#### 2020.11.02

1) 뉴스검색결과 전체 건수 항목 사라져서 삭제

#### 2020.05.02
1) 시작날짜, 종료날짜 입력하여 실행하도록 설정

2) column단위 병합에서 row 단위 개별 추가로 변경

In [1]:
import re
import time
import requests
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from tqdm import tqdm_notebook

In [4]:
#수집 : keyword/ 정렬순서(최신/과거)/ 날짜 지정 
def searchNewsExtractor(keyword,sdate,edate): #자동
    df = pd.DataFrame(columns = ["날짜", "언론사", "제목", "본문", "url"])
    error_cnt= 0
    
    dot_sdate = sdate[0:4]+"."+sdate[4:6]+"."+sdate[6:8]
    dot_edate = edate[0:4]+"."+edate[4:6]+"."+edate[6:8]
    
    for i in tqdm_notebook(range(28)): #4,000건이 최대
        pageNo= 10*i+1
        url = "https://search.naver.com/search.naver?&where=news&query="+keyword+"&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=3&ds="+dot_sdate+"&de="+dot_edate+"&docid=&nso=so:dd,p:from"+sdate+"to"+edate+",a:all&mynews=0&start="+str(pageNo)+"&refresh_start=0"
        req = requests.get(url)
        html = req.text
        soup = BeautifulSoup(html, 'lxml')

        if soup.find("div",{"class":"not_found02"}):
            print(pageNo,"페이지  ->",soup.find("div",{"class":"not_found02"}))
            break 
        else:
            print(pageNo,"페이지  ->",soup.find("div",{"class":"not_found02"}))
            df = pd.concat([df, newsParser(soup, df)], ignore_index=True)
            
    return df


def newsParser(soup,df):
    news_boxs = soup.find_all("div",{"class":"news_area"})
   
    for box in news_boxs:
        press = box.find("a", {"class": "info press"}).text.replace("언론사 선정","").strip() #언론사
        url = box.find("a", {"class":"news_tit"})['href'] #url
        title = box.find("a", {"class":"news_tit"}).text.strip() #제목
        date = box.find_all("span", {"class": "info"}) #날짜
        content = box.find("div", {"class":"news_dsc"}).text.strip() #내용
    
        #-시간/-일 전 정제하기
        if len(date) ==1: 
            date = date[0].text
        else: 
            date = date[1].text
            
        try:
            match = re.search(r'\d{4}.\d{2}.\d{2}', date)
            date = str(match.group())
        except AttributeError:
            try: 
                if "일" in date:
                    match = re.search('\d+일 전', date)
                    date = str(match.group())
                elif "시간" in date:
                    match = re.search('\d+시간 전', date)
                    date = str(match.group())
                else: pass
            except AttributeError: 
                pass
            
        print("press:{} \n \n href:{} \n \n title:{} \n \n date:{} \n \n content:{}".format(press,url, title, date, content))
        row = pd.Series([date, press, title, content, url], index = ["날짜", "언론사", "제목", "본문", "url"])
        df = df.append(row, ignore_index=True)
        
    return df
    
def to_excel(filenm):

    for i in range(len(df)):
        #최신날짜 "-일전/-시간 전" 정제하기 
        if "일" in df['날짜'][i]:
            match = re.search('\d+', df['날짜'][i])
            now = datetime.now() #현재 시간(수집 시간)
            diff = int(match.group()) #몇일 전/몇시간 전인지
            date = now - timedelta(days=diff)
            df['날짜'][i] = date.strftime("%Y.%m.%d")

        elif "시간" in df['날짜'][i]:
            match = re.search('\d+', df['날짜'][i])
            now = datetime.now() #현재 시간(수집 시간)
            diff = int(match.group()) #몇일 전/몇시간 전인지
            date = now - timedelta(hours=diff)
            df['날짜'][i] = date.strftime("%Y.%m.%d")
            #print("몇시간전? :",diff,"\n현재시각: ",now, "\n바뀐 시간: ", date)
            #print("입력시간: ",date.strftime("%Y.%m.%d"))
        else: pass
    
    df.to_excel(filenm, index=True)
    return df

In [5]:
#####-------------main-----------------#####
keyword = "카운터포인트리서치" #카운터포인트리서치/스트래티지애널리틱스
sortNo = 1 #1:최신순, 2: 과거순
sdate = "20201001" #시작 날짜
edate = "20201030" #종료 날짜 
filenm = keyword+"_"+sdate+"_"+edate+".xlsx" #저장할 파일명

#1. 최신순/과거순
#searchNewsExtractor1(keyword,sortNo)

#2. 날짜지정
df = searchNewsExtractor(keyword,sdate,edate)
to_excel(filenm)

1 페이지  -> None


UnboundLocalError: local variable 'df' referenced before assignment

In [4]:
df

NameError: name 'df' is not defined